# Ambit IE Data Internship Assignment

In [ ]:
pip install streamlit requests beautifulsoup4 pandas numpy


# Web Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_company_data(company_code):
    # Initial URL for consolidated data
    url = f"https://www.screener.in/company/{company_code}/consolidated/"
    response = requests.get(url)
    
    # If consolidated data is not available, fallback to standalone data
    if response.status_code != 200:
        url = f"https://www.screener.in/company/{company_code}/"
        response = requests.get(url)
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extracting PE ratio
    pe_element = soup.find('li', text=lambda x: x and 'PE' in x)
    current_pe = float(pe_element.text.split()[1]) if pe_element else None
    
    # Extracting RoCE (Return on Capital Employed)
    roce_element = soup.find('li', text=lambda x: x and 'RoCE' in x)
    roce_value = float(roce_element.text.split()[1][:-1]) if roce_element else None
    
    # Extracting growth data for different time periods
    growth_data = {}
    for period in ['TTM', '3Years', '5Years', '10Years']:
        growth_element = soup.find('li', text=lambda x: x and f'{period}' in x)
        growth_data[period] = float(growth_element.text.split()[-1][:-1]) if growth_element else None
    
    return current_pe, roce_value, growth_data


# DCF Model Calculation

In [ ]:
import numpy as np

def calculate_intrinsic_pe(cost_of_capital, roce, growth_rate, high_growth_years, fade_years, terminal_growth_rate, tax_rate=0.25):
    cash_flows = []
    for year in range(1, high_growth_years + fade_years + 1):
        if year <= high_growth_years:
            growth = growth_rate
        else:
            fade_factor = (1 - ((year - high_growth_years) / fade_years))
            growth = terminal_growth_rate + (growth_rate - terminal_growth_rate) * fade_factor
        roce_cash_flow = roce * (1 - tax_rate)  # Adjusted for tax
        cash_flows.append(roce_cash_flow * (1 + growth))
    
    # Discount cash flows to present value
    discounted_cf = [cf / ((1 + cost_of_capital) ** i) for i, cf in enumerate(cash_flows, start=1)]
    intrinsic_value = sum(discounted_cf)
    
    return intrinsic_value  # This is the intrinsic PE derived from DCF calculation


# Streamlit Web 

In [ ]:
pip install --upgrade streamlit


In [ ]:
import warnings
warnings.filterwarnings("ignore", message="missing ScriptRunContext")


In [ ]:
import streamlit as st
import pandas as pd
import requests
from bs4 import BeautifulSoup

st.title("DCF Valuation Tool")

# User input for company symbol and financial parameters
symbol = st.text_input("Enter NSE/BSE Symbol", value="NESTLEIND")
cost_of_capital = st.slider("Cost of Capital (%)", 5.0, 15.0, 10.0) / 100
roce = st.slider("Return on Capital Employed (RoCE) (%)", 10.0, 30.0, 20.0) / 100
growth_rate = st.slider("High Growth Rate (%)", 5.0, 20.0, 10.0) / 100
high_growth_years = st.slider("High Growth Period (Years)", 5, 20, 15)
fade_years = st.slider("Fade Period (Years)", 5, 20, 15)
terminal_growth_rate = st.slider("Terminal Growth Rate (%)", 1.0, 5.0, 2.0) / 100

# Function to fetch data from screener.in
def fetch_company_data(symbol):
    url = f"https://www.screener.in/company/{symbol}/consolidated/"
    response = requests.get(url)
    if response.status_code != 200:
        url = f"https://www.screener.in/company/{symbol}/"
        response = requests.get(url)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract PE ratio
    try:
        pe_element = soup.find('li', string=lambda text: text and 'PE' in text)
        current_pe = float(pe_element.text.split()[1]) if pe_element else None
    except:
        current_pe = None
    
    # Extract RoCE
    try:
        roce_element = soup.find('li', string=lambda text: text and 'RoCE' in text)
        roce_value = float(roce_element.text.split()[1].strip('%')) if roce_element else None
    except:
        roce_value = None
    
    # Extract growth data
    growth_data = {}
    try:
        for period in ['TTM', '3Years', '5Years', '10Years']:
            growth_element = soup.find('li', string=lambda text: text and period in text)
            growth_data[period] = float(growth_element.text.split()[-1].strip('%')) if growth_element else None
    except:
        growth_data = None
    
    # Extract market cap and net profit for FY23
    try:
        market_cap_element = soup.find('li', string=lambda text: text and 'Market Cap' in text)
        market_cap = float(market_cap_element.text.split()[2].replace(',', '')) * 1e7 if market_cap_element else None

        net_profit_element = soup.find('li', string=lambda text: text and 'Net Profit' in text)
        net_profit = float(net_profit_element.text.split()[-2].replace(',', '')) * 1e7 if net_profit_element else None

        fy23_pe = market_cap / net_profit if market_cap and net_profit else None
    except:
        fy23_pe = None

    return current_pe, roce_value, growth_data, fy23_pe

# Function to calculate intrinsic PE
def calculate_intrinsic_pe(k_cost, roce, g_high, fade_years, fade_period, g_terminal):
    years = fade_years + fade_period
    pe_values = []
    
    for year in range(fade_years):
        growth_rate = g_high - (g_high - g_terminal) * year / fade_years
        roce_effect = roce * ((1 - g_terminal) / (k_cost - g_terminal))
        intrinsic_pe_year = roce_effect / ((k_cost - growth_rate) + 1e-8)
        pe_values.append(intrinsic_pe_year)
    
    fade_pe = sum(pe_values) / len(pe_values)
    return fade_pe

# Fetch data and perform calculations
if st.button("Fetch Data"):
    current_pe, roce_value, growth_data, fy23_pe = fetch_company_data(symbol)
    
    if current_pe and roce_value and fy23_pe:
        st.write(f"Current PE: {current_pe}")
        st.write(f"5-yr median RoCE: {roce_value}")
        st.write(f"Growth Data: {growth_data}")
        st.write(f"FY23 PE: {fy23_pe}")
        
        intrinsic_pe = calculate_intrinsic_pe(cost_of_capital, roce_value, growth_rate, high_growth_years, fade_years, terminal_growth_rate)
        st.write(f"Calculated Intrinsic PE: {intrinsic_pe:.2f}")
        
        degree_of_overvaluation = ((min(current_pe, fy23_pe) / intrinsic_pe) - 1) * 100
        st.write(f"Degree of Overvaluation: {degree_of_overvaluation:.2f}%")
    else:
        st.write("Unable to fetch complete data. Please check the input symbol.")
